<a href="https://colab.research.google.com/github/jehee-lee/jjtech/blob/main/mnist_CAM_RPi_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모델 생성, 모델 매개변수 불러오기

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

device = torch.device('cpu')
model = Net()
PATH = './model/'
model.load_state_dict(torch.load(PATH + 'model_for_RO_ANN.pt', map_location=device))
model.eval()

하드웨어 핀 설정 (GPIO)

및 비디오 설정

In [ ]:
import RPi.GPIO as GPIO
import cv2
from time import sleep as sl

led = 21
button = 16
GPIO.setmode(GPIO.BCM)

GPIO.setup(led,GPIO.OUT)
GPIO.setup(button,GPIO.IN,pull_up_down=GPIO.PUD_UP)
cap = cv2.VideoCapture(0)

frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
         int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

print(frame_size)


콜백함수 지정 및 인터럽트 실행문 선언

콜백함수는 버튼이 눌려지면 (풀업저항이 RISING될 때) 실행됨

콜백함수 구성)
*   led High
*   현재 이미지 저장
*   저장된 이미지 불러오기
*   led Low
*   이미지 전처리(이진화, 리사이징)
*   이미지 예측 및 이미지 출력


In [ ]:
def call_back_for_capture(channel):
    print("button pressed")
    GPIO.output(led,GPIO.HIGH)
    cv2.imwrite('./data/mnist.jpg',frame1)
    sl(1)
    img = cv2.imread('./data/mnist.jpg', cv2.IMREAD_GRAYSCALE)
    GPIO.output(led,GPIO.LOW)
    _, img1 = cv2.threshold(img,160, 255, cv2.THRESH_BINARY_INV) #threshold값은 주변 환경에 따라 수시로 수정해줄 것
    img2 = cv2.resize(img1, (28,28), interpolation = cv2.INTER_AREA)
    img3=torch.from_numpy(img2)
    result = img3.view(-1,28*28).float().to(device)
    single_prediction = model(result)
    print('Prediction:', torch.argmax(single_prediction,1).item())
    plt.imshow(img3.view(28,28),cmap='Greys')
    plt.show()

GPIO.add_event_detect(button, GPIO.RISING,
                      callback=call_back_for_capture)

스트림 실행
(esc를 누르면 종료)

In [ ]:
while(cap.isOpened()):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    frame = frame[500:1500,800:1800]
    frame1 = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    if (ret):
        frame2 = cv2.resize(frame1, dsize=(640, 480), interpolation=cv2.INTER_AREA)
        cv2.imshow('stream',frame2)
        cv2.moveWindow('stream',200,100)
        k = cv2.waitKey(1) & 0xFF
        if(k== 27):
            break;
cap.release()
cv2.destroyAllWindows()
GPIO.cleanup